# Demo: شبیه‌سازی RLPF روی متن تبلیغ

این نوت‌بوک از داده‌های مصنوعی (یا داده‌ی استخراج‌شده از Ad Library API) برای ساخت یک مدل پاداش و شبیه‌سازی انتخاب بهترین نسخه‌ی متن استفاده می‌کند.

In [ ]:
import pandas as pd
from src.reward_model import train_reward_model
from src.rlpf_sim import simulate_rlpf

df = pd.read_csv('../data/synthetic_ads.csv')
df.head()

In [ ]:
df['text'] = df['ad_creative_body'].fillna('')
df['score'] = df['ctr'].fillna(0)
reward_model = train_reward_model(df['text'], df['score'])
results = simulate_rlpf(df['text'].tolist()[:50], reward_model, n_variations=5)
results[:3]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

deltas = np.array([r['delta'] for r in results])
plt.hist(deltas, bins=20)
plt.title('Distribution of Reward Improvements')
plt.xlabel('Delta')
plt.ylabel('Count')
plt.show()